# Day 1 - Lab 2: Generating a Product Requirements Document (PRD) (Solution)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Introduction:**
This solution notebook demonstrates how to synthesize detailed, low-level requirements (user stories) into a high-level planning document (the PRD). It also introduces the advanced concept of using code (Pydantic models) to define and validate the structure of documentation.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We begin by setting up our environment and loading the key artifact from Lab 1: `day1_user_stories.json`. The `load_artifact` helper function reads the file content, and `json.loads` parses the JSON string into a Python list of dictionaries, making it ready for use in our prompts.

In [6]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="claude-sonnet-4-20250514")

# Load the artifact from Lab 1
user_stories_str = load_artifact("artifacts/day1_user_stories.json")
if user_stories_str:
    user_stories_data = json.loads(user_stories_str)
else:
    print("Warning: Could not load user stories. Lab may not function correctly.")
    user_stories_data = []

✅ LLM Client configured: Using 'anthropic' with model 'claude-sonnet-4-20250514'


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a Simple PRD

**Explanation:**
This prompt serves as a baseline. We provide the LLM with the user stories and ask it to summarize them into a few key sections. This is a simple synthesis task that demonstrates the LLM's ability to extract and reorganize information.

In [8]:
simple_prd_prompt = f"""
You are a Product Manager writing a Product Requirements Document (PRD) for a new hire onboarding tool.

Use the following JSON data containing user stories as your primary source of information:
<user_stories>
{user_stories_str}
</user_stories>

Generate a PRD in markdown format with the following sections:
1. **Introduction:** A brief overview of the project's purpose.
2. **User Personas:** A summary of the key users involved.
3. **Features / User Stories:** A list of the user stories and their acceptance criteria.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
# Product Requirements Document: New Hire Onboarding Tool

## Introduction

The New Hire Onboarding Tool is a comprehensive digital platform designed to streamline and enhance the employee onboarding experience. This tool addresses the critical need for a structured, trackable, and interactive onboarding process that benefits both new employees and the organizations welcoming them.

The primary purpose of this tool is to:
- Provide new hires with a clear, guided pathway through their initial weeks at the company
- Enable HR managers and team leaders to monitor progress and provide targeted support
- Create a feedback loop for continuous improvement of the onboarding process
- Ensure consistency and completeness in the onboarding experience across all departments

By centralizing onboarding activities, training materials, and progress tracking, this tool will reduce time-to-productivity for new employees while improving retention rates and overall employee 

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Explanation:**
This is a more advanced and practical task. Providing a template gives us much greater control over the final output. The LLM's task shifts from creative writing to structured content generation. We instruct it to fill in every section, which forces it to infer logical content for sections like "Success Metrics" and "Out of Scope" based on the provided requirements. This is a powerful pattern for creating consistent documentation.

In [10]:
# Load the PRD template from the 'templates' directory.
prd_template_content = load_artifact("templates/prd_template.md")

template_prd_prompt = f"""
You are a Senior Product Manager responsible for creating a detailed and formal Product Requirements Document (PRD).

Your task is to populate the provided PRD template using the information from the user stories JSON.

<prd_template>
{prd_template_content}
</prd_template>

<user_stories_json>
{user_stories_str}
</user_stories_json>

Fill out every section of the template. For sections like 'Success Metrics' or 'Out of Scope', you must infer reasonable content based on the user stories and the overall project goal of creating a new hire onboarding tool.
The final output should be the completed PRD in markdown format.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

--- Generating PRD from Template ---
# Product Requirements Document: Employee Onboarding Management Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Senior Product Manager |
| **Version** | 1.0 |
| **Last Updated** | December 2024 |

## 1. Executive Summary & Vision
*A high-level overview for stakeholders. What is this product, why are we building it, and what is the ultimate vision for its success?*

The Employee Onboarding Management Platform is a comprehensive digital solution designed to streamline and enhance the new hire experience while providing managers and HR teams with powerful tools to track, customize, and optimize the onboarding process. This platform addresses the fragmented nature of current onboarding processes by centralizing tasks, training materials, progress tracking, and feedback mechanisms into a single, intuitive interface. Our vision is to create a seamless onboarding experience that accelerates time-to-productivity for new hires while reducing 

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Explanation:**
This is the most advanced challenge. We are now using an LLM to write *code that validates documents*. Generating a Pydantic model turns our document's structure into a testable, code-based standard. This is a form of 'documentation-as-code' that allows for automated governance, ensuring all future PRDs conform to the same reliable format.

1.  **Prompting for Code:** We give the LLM the PRD template and ask it to generate a Pydantic model. Pydantic is a data validation library, and using it to define our document structure turns that structure into a testable, reusable standard.
2.  **Saving the Model:** We save the generated Python code to a specific location (`app/validation_models/prd_model.py`). This isn't just a temporary script; it's a formal part of our application's codebase, intended to be used for future validation tasks.
3.  **Saving the PRD:** Finally, we save the markdown PRD generated in the intermediate step. This becomes the official `day1_prd.md` artifact for our project.

In [12]:
pydantic_model_prompt = f"""
You are a Python developer specializing in data validation with Pydantic.

Based on the following markdown PRD template, generate a single Pydantic model class named `ProductRequirementsDocument` that represents its structure.

<prd_template>
{prd_template_content}
</prd_template>

The model should have fields that correspond to the main sections of the template. Use appropriate Python types (e.g., str, List, Dict) from the `typing` library.
Ensure you include the necessary imports from `pydantic` and `typing`.
Only output the raw Python code for the model, without any explanation.
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Use our standardized cleaning function
    cleaned_code = clean_llm_output(pydantic_model_code, language='python')
    
    print("\n--- Generated Pydantic Model ---")
    print(cleaned_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(cleaned_code, model_path)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge as our official artifact
if prd_from_template_output:
    save_artifact(prd_from_template_output, "artifacts/day1_prd.md")

--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
from pydantic import BaseModel
from typing import List, Dict, Optional
from datetime import date

class ProductRequirementsDocument(BaseModel):
    product_name: str
    status: str
    author: str
    version: str
    last_updated: date
    
    executive_summary_vision: str
    
    problem_statement: str
    user_personas_scenarios: List[str]
    
    goals_success_metrics: List[Dict[str, str]]
    
    functional_requirements_user_stories: str
    
    non_functional_requirements: List[str]
    
    release_plan_milestones: List[Dict[str, str]]
    
    out_of_scope_v1: List[str]
    future_work: List[str]
    
    open_questions: Optional[List[str]] = None
    dependencies: Optional[List[str]] = None
✅ Successfully saved artifact to: app/validation_models/prd_model.py
✅ Successfully saved artifact to: artifacts/day1_prd.md


## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.